# 这是一个用来模拟储能充放策略算法的模拟程序

![规则集说明图](./规则集-第一版.png)

In [41]:
from test_ruleset_battery import *
from pandas.core.frame import DataFrame
import pandas as pd
import os
from os.path import join

In [60]:
'''
模拟程序输入：df(p_pvlist, p_loadlist, c_outlist), c_pv, soc, soc_max, soc_min, p_bat_in_max, p_bat_out_max
分别对应的是：光伏a、负荷序列a，分时电价序列a，光伏上网电价，初始的SOC，SOC上下限，电池充上限功率，电池放上限功率

模拟程序输出：(p_batlist, p_上网list, p_下网list), modelist, soclist, targetlist
分别对应的是：策略a（电池充放功率，上网功率，下网功率），工作模式a，储能电池容量a, 经济效益指标a

---> 每一步策略生成之间包含一个soc的状态转移方程（类似）
 
带a的表示是时间序列格式的数据
不带a的表示不是时间序列，就是一个值

'''
def moc_program(df, c_pv, soc, soc_max, soc_min, p_bat_in_max, p_bat_out_max):
    
    times=df.shape[0]#获取行数，即知道需要循环执行几次调度算法，for i in range(1,times):执行一次调度算法，times+1
    
    #循环执行times次的调度算法，每次执行完毕之后做一下soc的计算，用于下一次循环的输入
    '''
    执行一次储能充放策略调度算法（可以是基于规则集的也可以是基于线性规划的）：
    schedule_bat_rule((p_pv,p_load,c_out,c_pv,soc,soc_max,soc_min,p_bat_in_max,p_bat_out_max),return 
    return p_bat,p_上网,p_下网,mode,mode_dict[mode])
    '''     
    p_batlist=[]
    p_上网list=[]
    p_下网list=[]
    modelist=[]
    targetlist=[]
    soclist=[]
        
    for i in range(1, times+1):
        p_pv=df.iloc[i-1,1]
        p_load=df.iloc[i-1,2]
        c_out=df.iloc[i-1,3]
        
        p_bat_now=schedule_bat_rule(p_pv,p_load,c_out,c_pv,soc,soc_max,soc_min,p_bat_in_max,p_bat_out_max)[0]
        p_上网_now=schedule_bat_rule(p_pv,p_load,c_out,c_pv,soc,soc_max,soc_min,p_bat_in_max,p_bat_out_max)[1]
        p_下网_now=schedule_bat_rule(p_pv,p_load,c_out,c_pv,soc,soc_max,soc_min,p_bat_in_max,p_bat_out_max)[2]
        mode_now=schedule_bat_rule(p_pv,p_load,c_out,c_pv,soc,soc_max,soc_min,p_bat_in_max,p_bat_out_max)[3]
        
        p_batlist.append(p_bat_now)
        p_上网list.append(p_上网_now)
        p_下网list.append(p_下网_now)
        modelist.append(mode_now)

        soc=soc-p_bat_now*0.5
        
        soclist.append(soc)         
        
    return p_batlist, p_上网list, p_下网list, modelist, soclist, targetlist

# 测试1：1101-1107实际园区光伏负荷

## 准备测试所需的输入数据

In [56]:
df=pd.read_excel("C:/Users/79918/Desktop/新建文件夹/new_ems/doc/算法相关/【模拟数据算法小测-xt】.xlsx",sheet_name="模拟数据")
df=df.iloc[:,[0,1,2,3]]

In [57]:
df

,datetime,p_pv,p_load,c_out
0,2021-11-01 00:00:00,0,27.563387,0.266
1,2021-11-01 01:00:00,0,25.563076,0.266
2,2021-11-01 02:00:00,0,26.165248,0.266
3,2021-11-01 03:00:00,0,29.008954,0.266
4,2021-11-01 04:00:00,0,26.973848,0.266
...,...,...,...,...
163,2021-11-07 19:00:00,0,31.499202,0.888
164,2021-11-07 20:00:00,0,30.960550,0.888
165,2021-11-07 21:00:00,0,29.981028,0.538
166,2021-11-07 22:00:00,0,27.507491,0.266


In [58]:
df=df
c_pv=0.415
#soc=30
soc_max=95
soc_min=5
p_bat_in_max=20
p_bat_out_max=20

## 测试结果

In [61]:
#获取模拟程序的结果
for soc in range(0,101):
    soc=soc
    p_batlist=moc_program(df, c_pv, soc, soc_max, soc_min, p_bat_in_max, p_bat_out_max)[0]
    p_上网list=moc_program(df, c_pv, soc, soc_max, soc_min, p_bat_in_max, p_bat_out_max)[1]
    p_下网list=moc_program(df, c_pv, soc, soc_max, soc_min, p_bat_in_max, p_bat_out_max)[2]
    modelist=moc_program(df, c_pv, soc, soc_max, soc_min, p_bat_in_max, p_bat_out_max)[3]
    soclist=moc_program(df, c_pv, soc, soc_max, soc_min, p_bat_in_max, p_bat_out_max)[4]
    #targetlist=moc_program(df, c_pv, soc, soc_max, soc_min, p_bat_in_max, p_bat_out_max)[5]
    #对结果进行整理
    #把工作模式对应的简单文字说明加到results里面

    modetextlist=[]
    for i in modelist:
        modetextlist.append(mode_dict[i])

    results={
        "datetime":df.iloc[:,0].tolist(),
        "光伏功率_":df.iloc[:,1].tolist(),
        "负荷功率_":df.iloc[:,2].tolist(),
        "电价_":df.iloc[:,3].tolist(),
        "电池充放功率指令" : p_batlist,
        "上网功率" : p_上网list,
        "下网功率" : p_下网list,
        "工作模式" : modelist,
        "工作模式说明" : modetextlist,
        "下一步soc%" : soclist
    #    "经济效益指标" : targetlist
    }
    results=DataFrame(results)#将字典转换成为数据框

    #把整理完毕的结果输出看一下
    a = join(r'C:\\Users\\79918\\Desktop\\结果\\', str(soc)+'.csv')
    results.to_csv(a,encoding='utf_8_sig')

In [62]:
newdir = r'C:\\Users\\79918\\Desktop\\结果\\'
list = os.listdir(newdir)  # 列出文件夹下所有的目录与文件

writer = pd.ExcelWriter('SOC_1101-1107.xlsx')


for i in range(0,len(list)):
    a=join(newdir,list[i])
    data = pd.read_csv(a,encoding="utf_8_sig")
    data.to_excel(writer, sheet_name=list[i])

writer.save()

# 测试2：光伏弱的情况